In [4]:
# This is necessary to recognize the modules
import os
import sys
from decimal import Decimal

root_path = os.path.abspath(os.path.join(os.getcwd(), '../..'))
sys.path.append(root_path)

In [7]:
from data_sources.clob import CLOBDataSource

# Get trading rules and candles
clob = CLOBDataSource()

In [2]:
# Constants
CONNECTOR_NAME = "binance"
INTERVAL = "15m"
DAYS = 20
TOP_X_MARKETS = 10  # Number of top markets to select
# Assuming the `top_markets` DataFrame has been generated as shown in the previous steps
# Constants
TOTAL_AMOUNT = 3000  # General total amount for all markets
MIN_AMOUNT_PER_MARKET = 200  # Minimum amount per market
ACTIVATION_BOUNDS = 0.002  # Input activation bounds
MAX_OPEN_ORDERS = 1  # Input max open orders for each market

# Example values
TAKE_PROFIT_MULTIPLIER = 0.3  # Multiplier for take profit based on NATR
AMOUNTS_QUOTE_PCT = [0.1, 0.1, 0.1, 0.1, 0.2, 0.2]  # Weights for each cluster

# Features configuration
VOLATILITY_WINDOW = 100
TREND_SHORT_WINDOW = 20
TREND_LONG_WINDOW = 100
VOLUME_SHORT_WINDOW = 20
VOLUME_LONG_WINDOW = 100

# Download data
- Get trading rules
- Get candles for the last x days

In [9]:
import asyncio

trading_rules = await clob.get_trading_rules(CONNECTOR_NAME)
trading_pairs = trading_rules.filter_by_quote_asset("FDUSD")\
    .filter_by_min_notional_size(Decimal("5"))\
    .get_all_trading_pairs()

tasks = [clob.get_candles_last_days(
    connector_name=CONNECTOR_NAME,
    trading_pair=trading_pair,
    interval=INTERVAL,
    days=DAYS,
) for trading_pair in trading_pairs]

candles = await asyncio.gather(*tasks)
candles = {trading_pair: candle for trading_pair, candle in zip(trading_pairs, candles)}

2024-07-26 18:19:06,909 - asyncio - ERROR - Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x13a42a890>
2024-07-26 18:19:06,912 - asyncio - ERROR - Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x13a25cf40>, 15886.477836916)]']
connector: <aiohttp.connector.TCPConnector object at 0x13a42a860>
2024-07-26 18:19:07,523 - asyncio - ERROR - Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x12b201c60>
2024-07-26 18:19:08,920 - asyncio - ERROR - Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x13a3368f0>
2024-07-26 18:19:08,921 - asyncio - ERROR - Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x12a554c40>, 15888.474082875)]']
connector: <aiohttp.connector.TCPConnector object at 0x13a336950>
2024-07-26 18:19:08,937 - asyncio - ERROR - Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x13a39ec

In [10]:
from features.candles.trend import TrendConfig
from features.candles.volatility import VolatilityConfig
from features.candles.volume import VolumeConfig
from research_notebooks.dneitor.utils import generate_report

report = generate_report(
    candles=candles,
    volatility_config=VolatilityConfig(window=VOLATILITY_WINDOW),
    trend_config=TrendConfig(short_window=TREND_SHORT_WINDOW, long_window=TREND_LONG_WINDOW),
    volume_config=VolumeConfig(short_window=VOLUME_SHORT_WINDOW, long_window=VOLUME_LONG_WINDOW))
report

Error processing RENDER-FDUSD: unsupported operand type(s) for /: 'NoneType' and 'int'


,trading_pair,mean_volatility,mean_natr,mean_bb_width,latest_trend,average_volume_per_hour,normalized_score
0,USTC-FDUSD,0.00731984,0.00633831,7.8877229,0.0817414,4267.77236394,0.00011683
1,TIA-FDUSD,0.00706082,0.00847787,10.61859969,-0.26045596,15028.56937926,0.00055382
2,LINK-FDUSD,0.00385108,0.00523608,5.33370649,0.0637494,61701.08591921,0.00114294
3,LDO-FDUSD,0.00622596,0.00619089,7.30822578,-0.56266522,8641.93548811,0.00021923
4,LTC-FDUSD,0.00303772,0.00376106,3.95389369,-0.41167972,21186.32349076,0.00029089
...,...,...,...,...,...,...,...
87,CHZ-FDUSD,0.00421227,0.00260424,5.67165614,-0.80071257,2198.24942032,0.00004321
88,MANTA-FDUSD,0.00592397,0.00535685,7.97284695,-0.52671323,4408.00282189,0.0001219
89,ZRO-FDUSD,0.00734464,0.009554,10.20924606,-0.24403415,49610.60521492,0.00175869
90,ALT-FDUSD,0.00759691,0.00748437,9.17031892,-0.7029905,6577.67185972,0.00020935


In [3]:
from research_notebooks.dneitor.utils import filter_top_markets

top_markets = filter_top_markets(report_df=report, top_x=TOP_X_MARKETS, min_volume_usd=2000, min_atr=0.008, trend_threshold=-0.5)
top_markets

NameError: name 'report' is not defined

In [ ]:
from research_notebooks.dneitor.utils import generate_config, dump_dict_to_yaml

strategy_config = generate_config(
    id="dneitor-binance_0.4",
    connector_name=CONNECTOR_NAME,
    candles=candles,
    top_markets=top_markets,
    total_amount=TOTAL_AMOUNT,
    amounts_quote_pct=AMOUNTS_QUOTE_PCT,
    activation_bounds=ACTIVATION_BOUNDS,
    take_profit_multiplier=TAKE_PROFIT_MULTIPLIER,
    max_open_orders=MAX_OPEN_ORDERS,
    min_amount_per_market=MIN_AMOUNT_PER_MARKET,
)
dump_dict_to_yaml("configs/", strategy_config)